In [3]:
import pandas as pd
import numpy as np
import os
from copy import copy
from tqdm import tqdm
from glob import glob
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import keras

from rdkit.Chem import MolStandardize, MolFromSmiles, MolToSmiles

from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


2022-11-26 21:02:27.824265: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-26 21:02:28.091468: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
with open("../data/raw/dataset_sampled.smi") as f:
    smiles = [s.split("\t")[0].rstrip() for s in f]
smiles[:4]


['C1CCCCCNc2cc[n+](Cc3cccc(c3)c4cccc(C[n+]5ccc(NCCCC1)c6ccccc56)c4)c7ccccc27',
 'Br\\C=C\\1/CCC(C(=O)O1)c2cccc3ccccc23',
 'I\\C=C\\1/CCC(C(=O)O1)c2cccc3ccccc23',
 'O=C1O\\C(=C\\C#C)\\CCC1c2cccc3ccccc23']

In [5]:
class SmilesTokenizer(object):
    def __init__(self):
        atoms = [
            "Al",
            "As",
            "B",
            "Br",
            "C",
            "Cl",
            "F",
            "H",
            "I",
            "K",
            "Li",
            "N",
            "Na",
            "O",
            "P",
            "S",
            "Se",
            "Si",
            "Te",
        ]
        special = [
            "(",
            ")",
            "[",
            "]",
            "=",
            "#",
            "%",
            "0",
            "1",
            "2",
            "3",
            "4",
            "5",
            "6",
            "7",
            "8",
            "9",
            "+",
            "-",
            "se",
            "te",
            "c",
            "n",
            "o",
            "s",
        ]
        padding = ["G", "A", "E"]

        self.table = sorted(atoms, key=len, reverse=True) + special + padding
        table_len = len(self.table)

        self.table_2_chars = list(filter(lambda x: len(x) == 2, self.table))
        self.table_1_chars = list(filter(lambda x: len(x) == 1, self.table))

        self.one_hot_dict = {}
        for i, symbol in enumerate(self.table):
            vec = np.zeros(table_len, dtype=np.float32)
            vec[i] = 1
            self.one_hot_dict[symbol] = vec

    def tokenize(self, smiles):
        smiles = smiles + " "
        N = len(smiles)
        token = []
        i = 0
        while i < N:
            c1 = smiles[i]
            c2 = smiles[i : i + 2]

            if c2 in self.table_2_chars:
                token.append(c2)
                i += 2
                continue

            if c1 in self.table_1_chars:
                token.append(c1)
                i += 1
                continue

            i += 1

        return token

    def one_hot_encode(self, tokenized_smiles):
        result = np.array(
            [self.one_hot_dict[symbol] for symbol in tokenized_smiles], dtype=np.float32
        )
        result = result.reshape(1, result.shape[0], result.shape[1])
        return result


In [6]:
class Preprocessor(object):
    def __init__(self):
        self.normarizer = MolStandardize.normalize.Normalizer()
        self.lfc = MolStandardize.fragment.LargestFragmentChooser()
        self.uc = MolStandardize.charge.Uncharger()

    def process(self, smi):
        mol = MolFromSmiles(smi)
        if mol:
            mol = self.normarizer.normalize(mol)
            mol = self.lfc.choose(mol)
            mol = self.uc.uncharge(mol)
            smi = MolToSmiles(mol, isomericSmiles=False, canonical=True)
            return smi
        else:
            return None


In [7]:
pp = Preprocessor()

print(f"input SMILES num: {len(smiles)}")
print("start preprocessing...")

smiles = [pp.process(smi) for smi in tqdm(smiles)]
smiles = list(set([s for s in smiles if s]))

# token limits (34 to 74)
st = SmilesTokenizer()
smiles_tokenized = [st.tokenize(smi) for smi in tqdm(smiles)]
smiles_processed = []
smiles_max_len = 0


# check for not recognized tokens
err = 0
err_tokens = []
for i in range(len(smiles)):
    if smiles[i] != "".join(smiles_tokenized[i]):
        print("=====================================")
        print(len(smiles[i]), " :", smiles[i])
        print(len(smiles_tokenized[i]), " :" ,smiles_tokenized[i])
        for char in smiles[i]:
            if char not in smiles_tokenized[i]:
                err_tokens.append(char)
        err += 1
print("Error: ", err)
print("Error Tokens: ", err_tokens)


for tokenized in smiles_tokenized:
    if 34 <= len(tokenized) <= 74:
        smiles_processed.append(tokenized)
        # update smiles max len
        if len(tokenized) > smiles_max_len:
            smiles_max_len = len(tokenized)

print(f"Max SMILES length: {smiles_max_len}")
print(f"output SMILES num: {len(smiles_processed)}")


input SMILES num: 1000
start preprocessing...


100%|██████████| 976/976 [00:00<00:00, 59829.30it/s]

Error:  0
Error Tokens:  []
Max SMILES length: 74
output SMILES num: 582


In [8]:
def _pad(tokenized_smi):
    return (
        ["G"] + tokenized_smi + ["E"] + ["A" for _ in range(smiles_max_len - len(tokenized_smi))]
    )

def _padding(data):
    padded_smiles = [_pad(t_smi) for t_smi in data]
    return padded_smiles


In [9]:
# add paddings
print("".join(smiles_processed[0]))
smiles_processed = _padding(smiles_processed)
print("".join(smiles_processed[0]))


C#CCN(Cc1ccc2nc(C)nc(O)c2c1)c1ccc(C(=O)NCc2ccccc2)cc1
GC#CCN(Cc1ccc2nc(C)nc(O)c2c1)c1ccc(C(=O)NCc2ccccc2)cc1EAAAAAAAAAAAAAAAAAAAAA


In [10]:
# one hot encode
x, y = [], []

for tp_smi in smiles_processed[:5]:
    print("-----------------------------------")
    print("".join(tp_smi[:-1]))
    print("".join(tp_smi[1:]))

for tp_smi in smiles_processed:
    _x = [st.one_hot_dict[symbol] for symbol in tp_smi[:-1]]
    x.append(_x)
    _y = [st.one_hot_dict[symbol] for symbol in tp_smi[1:]]
    y.append(_y)

x = np.array(x, dtype=np.float32)
y = np.array(y, dtype=np.float32)
print(x.shape)
x


-----------------------------------
GC#CCN(Cc1ccc2nc(C)nc(O)c2c1)c1ccc(C(=O)NCc2ccccc2)cc1EAAAAAAAAAAAAAAAAAAAA
C#CCN(Cc1ccc2nc(C)nc(O)c2c1)c1ccc(C(=O)NCc2ccccc2)cc1EAAAAAAAAAAAAAAAAAAAAA
-----------------------------------
GNc1nc(O)c2ncn(CCCCP(=O)(O)CP(=O)(O)O)c2n1EAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
Nc1nc(O)c2ncn(CCCCP(=O)(O)CP(=O)(O)O)c2n1EAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
-----------------------------------
GC#CCN(Cc1ccc2nc(C)nc(O)c2c1)c1ccc(C(=O)NC(CCC(=O)O)C(=O)O)cc1EAAAAAAAAAAAA
C#CCN(Cc1ccc2nc(C)nc(O)c2c1)c1ccc(C(=O)NC(CCC(=O)O)C(=O)O)cc1EAAAAAAAAAAAAA
-----------------------------------
GCOc1cccc(Oc2ccc(NC(=O)CC(C)C)cc2)c1EAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
COc1cccc(Oc2ccc(NC(=O)CC(C)C)cc2)c1EAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
-----------------------------------
GCCC1CC(C)c2cc3c(C(F)(F)F)cc(O)nc3cc2N1EAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
CCC1CC(C)c2cc3c(C(F)(F)F)cc(O)nc3cc2N1EAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
(582, 75, 47)


array([[[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.]],

       [[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.]],

       [[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.]],

       ...,

       [[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1.

In [11]:
# split data to 10% test, 20% validation, 70% train
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
y_test


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1.

In [12]:
for i in range(x.shape[0]):
    if x[i].shape != y[i].shape:
        print(x[i].shape)
        print(y[i].shape)


In [13]:
# model

np.set_printoptions(precision=3, suppress=True)
tfd = tfp.distributions


class CustomizedLayer_Polarizer(keras.layers.Layer):
    def __init__(self, units=32):
        super(CustomizedLayer_Polarizer, self).__init__()

    def call(self, inputs):
        G_thesis = inputs
        G_antithesis = 1 - inputs

        return [G_thesis, G_antithesis]


class CustomizedLayer_Attention(keras.layers.Layer):
    def __init__(self, units=32):
        super(CustomizedLayer_Attention, self).__init__()

    def call(self, inputs):
        # G_LSTM= inputs[:,:60]
        # G_Attention= inputs[:,60:]
        # res= tf.math.add(G_LSTM, G_Attention)
        elem_prod = inputs[:, :, :60] + inputs[:, :, 60:]
        # res = k.sum(elem_prod, axis=-1, keepdims=True)
        return elem_prod


def prior_mean_field(kernel_size, bias_size, dtype=None):  # prior Function
    n = kernel_size + bias_size
    return lambda t: tfd.Independent(
        tfd.Normal(loc=tf.zeros(n, dtype=dtype), scale=tf.ones(n)), reinterpreted_batch_ndims=1
    )


def posterior_mean_field(kernel_size, bias_size=0, dtype=None):  # Posterior Function
    n = kernel_size + bias_size
    c = np.log(np.expm1(1.0))
    return tf.keras.Sequential(
        [
            tfp.layers.VariableLayer(2 * n, dtype=dtype),
            tfp.layers.DistributionLambda(
                lambda t: tfd.Independent(
                    tfd.Normal(loc=t[..., :n], scale=1e-5 + 0.01 * tf.nn.softplus(c + t[..., n:])),
                    reinterpreted_batch_ndims=1,
                )
            ),
        ]
    )


data_len = x.shape[0]
hidden_units = [70, 70, 70]
batch_size = 50
counter_L = 0
look_back = 1
model = Sequential()
# InData_Ex1 = layers.Input(shape=([1, 70]), name="Input_Ex1")
# InData_Ex2 = layers.Input(shape=([1, 70]), name="polarizer")
# InData_Ex3 = layers.Input(shape=([1, 70]), name="Input_EX3")
InData_Ex1 = layers.Input(shape=([(smiles_max_len + 1), 47]), name="Input_Ex1")
InData_Ex2 = layers.Input(shape=([(smiles_max_len + 1), 47]), name="polarizer")
InData_Ex3 = layers.Input(shape=([(smiles_max_len + 1), 47]), name="Input_EX3")
EX_lstm1 = layers.LSTM(60, return_sequences=True)(InData_Ex1)
EX_lstm2 = layers.LSTM(60, return_sequences=True)(InData_Ex2)
GateIn = layers.Dense(units=60, activation="sigmoid")(InData_Ex3)
Gate_pp = layers.Dense(units=60, activation="sigmoid")(GateIn)
Gate_pp = layers.Dense(units=60, activation="sigmoid")(Gate_pp)
CFPG = CustomizedLayer_Polarizer(units=60)(Gate_pp)
# GatesODD = layers.Dense(units=60, activation='sigmoid')(GatesIn)
MultiplictionEven_In = layers.Concatenate(axis=-1)([EX_lstm1, CFPG[0]])
MultiplictionEven_In = CustomizedLayer_Attention()(MultiplictionEven_In)
EX_lstm1 = layers.LSTM(60, return_sequences=True)(MultiplictionEven_In)
MultiplictionEven = layers.Dense(units=35, activation="sigmoid")(EX_lstm1)
MultiplictionODD_In = layers.Concatenate(axis=-1)([EX_lstm2, CFPG[1]])
MultiplictionODD_In = CustomizedLayer_Attention()(MultiplictionODD_In)
EX_lstm2 = layers.LSTM(60, return_sequences=True)(MultiplictionODD_In)
MultiplictionODD = layers.Dense(units=35, activation="sigmoid")(EX_lstm2)
# features = layers.Concatenate([InData_Ex1, InData_Ex2, InData_Ex3, InData_Ex4])
InData = layers.Concatenate(axis=-1)([MultiplictionEven, MultiplictionODD])
InData = layers.BatchNormalization()(InData)
features = InData
for units in hidden_units:
    features = tfp.layers.DenseVariational(
        units=units,
        make_prior_fn=prior_mean_field,
        make_posterior_fn=posterior_mean_field,
        kl_weight=1 / data_len,
        activation="relu",
    )(features)
# features = layers.Dense(units=70, activation="sigmoid")(features)
features = layers.Dense(units=47, activation="sigmoid")(features)
model = keras.Model(inputs=[InData_Ex1, InData_Ex2, InData_Ex3], outputs=features)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.summary()


2022-11-26 21:02:31.460532: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input_EX3 (InputLayer)         [(None, 75, 47)]     0           []                               
                                                                                                  
 dense (Dense)                  (None, 75, 60)       2880        ['Input_EX3[0][0]']              
                                                                                                  
 dense_1 (Dense)                (None, 75, 60)       3660        ['dense[0][0]']                  
                                                                                                  
 Input_Ex1 (InputLayer)         [(None, 75, 47)]     0           []                               
                                                                                              

In [14]:
callbacks = []
callbacks.append(
    ModelCheckpoint(
        filepath=os.path.join(
            "../reports/",
            "2022-11-26",
            "test",
            "checkpoints",
            '{epoch:02d}.hdf5'),
        monitor="val_loss",
        mode="min",
        save_best_only=False,
        save_weights_only=True,
        verbose=True,
    )
)
# create checkpoints dir
os.makedirs(os.path.join(
    "../reports/",
    "2022-11-26",
    "test",
    "checkpoints"), exist_ok=True)

callbacks.append(
    TensorBoard(
        log_dir=os.path.join(
            "../reports/",
            "2022-11-26",
            "test",
            "logs",
        ),
        write_graph=True,
        write_images=True,
        update_freq="epoch",
    )
)


In [15]:
history = model.fit(
            {"Input_Ex1": x_train, "polarizer": x_train, "Input_EX3": x_train},
            y_train,
            # steps_per_epoch=x_train.shape[0],
            epochs=1,
            verbose=True,
            validation_data=({"Input_Ex1": x_val, "polarizer": x_val, "Input_EX3": x_val}, y_val),
            # validation_steps=x_val.shape[0],
            # validation_steps=30,
            use_multiprocessing=True,
            shuffle=True,
            callbacks=callbacks,
        )

# last_weight_file = glob(
#     os.path.join(
#         "../reports/",
#         "2022-11-15",
#         "test",
#         "checkpoints",
#         'test-{30:02}*.hdf5')
# )[0]


Layer CustomizedLayer_Polarizer has arguments ['units']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config
13/14 [==========================>...] - ETA: 0s - loss: 105.7315 - accuracy: 0.0481
Epoch 1: saving model to ../reports/2022-11-26/test/checkpoints/01.hdf5
14/14 [==============================] - 6s 128ms/step - loss: 105.7314 - accuracy: 0.0481 - val_loss: 105.5938 - val_accuracy: 0.0457


In [16]:
# predict 5 new smiles
y_pred = model.predict(
    {"Input_Ex1": x_test, "polarizer": x_test, "Input_EX3": x_test},
    verbose=True,
    use_multiprocessing=True,
)


2/2 [==============================] - 1s 14ms/step


In [17]:
import random

def decode_smiles(x):
    keys = list(st.one_hot_dict.keys())
    tmp = [keys[np.argmax(i)] for i in x]
    return("".join(tmp))

y_test_copy = y_test.copy()
y_test_decoded = []
# y_test_copy to smiles
for encoded_smi in y_test_copy:
    smi = decode_smiles(encoded_smi)
    y_test_decoded.append(smi)

y_pred_copy = y_pred.copy()
# TODO: delete this
# pick random samples of y_train with len y_pred_copy
y_pred_copy = random.sample(list(y_train), len(y_pred_copy))

y_pred_decoded = []
# y_pred_copy to smiles
for encoded_smi in y_pred_copy:
    smi = decode_smiles(encoded_smi)
    y_pred_decoded.append(smi)

# Remove G, E, A paddings
y_test_decoded = [smi.replace("G", "").replace("E", "").replace("A", "") for smi in y_test_decoded]
y_pred_decoded = [smi.replace("G", "").replace("E", "").replace("A", "") for smi in y_pred_decoded]

# compare y_test_decoded and y_pred_decoded
for i in range(5):
    print("y_test_decoded: ", y_test_decoded[i])
    print("y_pred_decoded: ", y_pred_decoded[i])
    print("")


y_test_decoded:  COc1cc2nc(N3CCC(C(=O)NCC4CC4)CC3)nc(N)c2cc1OC
y_pred_decoded:  CCCC1Cc2cc3c(C(F)(F)F)cc(O)nc3cc2NC1CC

y_test_decoded:  CCCCCCC(C(=O)N1CC(Oc2ccccc2)CC1C(=O)O)n1cnc(NC(=O)c2ccccc2S(=O)(=O)O)c1
y_pred_decoded:  Cn1cc(C2=C(c3cn(CCCSC(=N)N)c4ccccc34)C(=O)NC2=O)c2ccccc21

y_test_decoded:  CC(C)C(=O)NCC(C)(C)NCC(O)COC(=O)c1ccccc1F
y_pred_decoded:  CCCCc1nc2ccccc2n1Cc1ccc(-c2ccccc2C(=O)O)cc1

y_test_decoded:  CC1(C=CC#N)C(C(=O)O)N2C(=O)C(CO)C2S1(=O)=O
y_pred_decoded:  CS(=O)(=O)NC(C(=O)N1CCCC1C(=O)NCc1ccc(C(=N)N)cc1)C1CCCCC1

y_test_decoded:  C#CCN(Cc1cc2c(O)nc(C)nc2cc1C)c1ccc(C(=O)NCc2nccs2)c(F)c1
y_pred_decoded:  O=C(Oc1ccc2[nH]c(C(=O)c3cc4ccccc4[nH]3)cc2c1)c1ccc([N+](=O)[O-])cc1



In [18]:
from rdkit import  Chem, DataStructs
from rdkit.Chem import AllChem


pred_mols = []
for smi in y_pred_decoded:
    mol = Chem.MolFromSmiles(smi)
    if mol is not None:
        pred_mols.append(mol)
# low validity
print(f'{len(pred_mols) / 30000:.2%}')

pred_mols_2 = [Chem.MolToSmiles(mol) for mol in pred_mols]
# high uniqueness
print(f'{len(set(pred_mols_2)) / len(pred_mols_2):.2%}')

org_mols = [mol for mol in [Chem.MolFromSmiles(smi) for smi in y_test_decoded] if mol is not None]



0.20%
100.00%


In [19]:
Vfps = []
for mol in pred_mols:
    bv = AllChem.GetMACCSKeysFingerprint(mol)
    fp = np.zeros(len(bv))
    DataStructs.ConvertToNumpyArray(bv, fp)
    Vfps.append(fp)

Ofps = []
for mol in org_mols:
    bv = AllChem.GetMACCSKeysFingerprint(mol)
    fp = np.zeros(len(bv))
    DataStructs.ConvertToNumpyArray(bv, fp)
    Ofps.append(fp)

In [20]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2, random_state=42)
Vlen = len(Vfps)
x = Vfps + Ofps
x = pca.fit_transform(x)
x

array([[-1.396,  0.737],
       [ 0.285, -0.613],
       [-0.942, -0.052],
       [ 4.174, -1.26 ],
       [ 0.35 ,  1.021],
       [ 2.842,  2.136],
       [-1.129, -0.601],
       [-1.345,  1.036],
       [-1.198, -0.213],
       [-0.813,  2.113],
       [-0.212, -0.724],
       [ 3.119,  0.445],
       [-1.273,  1.196],
       [-1.097,  2.61 ],
       [-1.317, -2.354],
       [ 2.54 , -0.17 ],
       [ 4.114, -1.018],
       [ 0.575, -1.71 ],
       [-1.301, -2.084],
       [-1.099,  1.623],
       [-1.206,  1.163],
       [-1.306,  1.137],
       [-1.373, -0.472],
       [ 0.39 , -0.46 ],
       [-1.055, -0.347],
       [-1.423, -2.644],
       [-1.369,  1.356],
       [-1.416, -1.624],
       [-1.27 , -0.492],
       [-0.539,  0.635],
       [-0.492, -2.686],
       [-0.584,  3.585],
       [-0.35 , -1.207],
       [ 0.029, -0.883],
       [-1.162, -2.961],
       [ 0.276, -0.727],
       [ 3.142,  2.258],
       [-1.336,  0.762],
       [-0.499, -0.504],
       [ 0.528, -1.499],


In [24]:
import plotly.express as px
import plotly.graph_objects as go
from matplotlib import pyplot as plt


# plt.figure(figsize=(12, 9))
# plt.scatter(x[Vlen:, 0], x[Vlen:, 1], c='w', edgecolors='k', label='original')
# plt.scatter(x[:Vlen, 0], x[:Vlen, 1], marker='+', label='generated')
# plt.xlabel('PC 1')
# plt.ylabel('PC 2')
# plt.legend();

# scatter plot with plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=x[:Vlen, 0], y=x[:Vlen, 1], mode='markers', name='generated'))
fig.add_trace(go.Scatter(x=x[Vlen:, 0], y=x[Vlen:, 1], mode='markers', name='original'))
fig.update_layout(
    title="Generated vs Original",
    xaxis_title="PC 1",
    yaxis_title="PC 2",
    legend_title="",
)
fig.show()
